In [40]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AudioProcessor

In [29]:
db_conn = db.get_connection()
video_id = "7490975051978820886"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [30]:
video_url, video_path, audio_path = download.tiktok_full(video_id)


INFO - [EnrichmentWorker] - Downloading MP4 for 7490975051978820886 → /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7490975051978820886.mp4
INFO - [EnrichmentWorker] - Audio 7490975051978820886 is already downloaded in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7490975051978820886.wav. Skipping


In [31]:
model_name = "ggml-large-v3.bin" 
transcript, lang, output = transcribe_audio(audio_path, model_name)
logging.info(transcript)

INFO - [EnrichmentWorker] - Starting transcriber model ggml-large-v3.bin for audio in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7490975051978820886.wav
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO - [EnrichmentWorker] - Pentru ăștia cu turul 2 înapoi, uite problemele reale.
 Mă bucur pentru voi că strigați prin stradă și că vă uitați la unul care se cațără prin copaci.
 Asta, Ucraina lui aprilie 2025.
 N-avem democrație la noi în țară, n-avem suveranitate.
 Să vină Rusia să ne salveze.
 Rusia care nu respectă suveranitatea Ucrainei și comite genocid, omorând copii.
 Aceeași Rusia pe care o să vezi cei care strigă acum pe străzi.
 Turul 2, înapoi.
 Turul 2, înapoi.
 Să vină Rus

In [32]:
sentiment_results = SentimentAnalyzer.get_sentiment(transcript, "ro")

INFO - [EnrichmentWorker] - SENTIMENT MODEL OUTPUT SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1580, -0.9222]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INFO - [EnrichmentWorker] - SENTIMENT MODEL SCORES [0.8889624  0.11103762]


In [33]:
sentiment_results

{'positive': 0.11103761941194534,
 'neutral': 0.0,
 'negative': 0.8889623880386353}

In [34]:
description = get_video_description(video_id, db_conn)

In [41]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7490975051978820886.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video features a man speaking directly to the camera, criticizing those who he claims are ignorant of the realities of the war in Ukraine. He contrasts the suffering in Ukraine, citing a Russian bombing in Kryvyi Rih that killed children, with the perceived political posturing of others. The video includes text overlays identifying the location as Kyiv, Ukraine, and referencing a 'round 2 back' and a date of April 4th. It also shows a Ukraini

In [42]:
analysis_result_json

{'summary': "The video features a man speaking directly to the camera, criticizing those who he claims are ignorant of the realities of the war in Ukraine. He contrasts the suffering in Ukraine, citing a Russian bombing in Kryvyi Rih that killed children, with the perceived political posturing of others. The video includes text overlays identifying the location as Kyiv, Ukraine, and referencing a 'round 2 back' and a date of April 4th. It also shows a Ukrainian flag flying at half-mast, indicating a day of mourning. The speaker urges viewers to share the message to spread awareness of the truth.",
 'identified_subjects': [{'subject': 'Ukraine', 'stance': 1.0},
  {'subject': 'Russia', 'stance': -1.0},
  {'subject': 'Democracy', 'stance': -1.0},
  {'subject': 'National Sovereignty', 'stance': -1.0},
  {'subject': 'War', 'stance': -1.0},
  {'subject': 'Vladimir Putin', 'stance': -1.0}],
 'overall_alignment': 0.5}

In [39]:
analysis_result_json

{'summary': "The video features a man speaking directly to the camera, criticizing those who he believes are not facing reality regarding the war in Ukraine. He contrasts the suffering in Ukraine, citing a bombing in Kryvyi Rih that killed children, with what he perceives as a lack of focus on real issues by some people in Romania. He sarcastically suggests Russia should 'save' Romania and criticizes those who praise Russia while claiming their own country lacks democracy and sovereignty. The video also shows footage of a Ukrainian flag flying at half-mast, indicating a day of mourning.",
 'identified_subjects': [{'subject': 'Ukraine', 'stance': 0.8},
  {'subject': 'Russia', 'stance': -1.0},
  {'subject': 'Democracy', 'stance': -0.5},
  {'subject': 'National Sovereignty', 'stance': -0.5},
  {'subject': 'War', 'stance': -0.8},
  {'subject': 'Vladimir Putin', 'stance': -1.0}],
 'overall_alignment': 0.6}

In [36]:
analysis_result_json

{'summary': 'The video features a man speaking directly to the camera, with a backdrop of a damaged apartment building and trees. He criticizes people who complain about a lack of democracy and sovereignty in Romania while praising those who do not respect the sovereignty of other countries. He references Ukraine, mentioning a bombing in Kryvyi Rih on April 4th and a day of mourning. The speaker emphasizes the importance of people seeing the truth about what is happening in Ukraine and encourages sharing the message.',
 'identified_subjects': [{'subject': 'Ukraine', 'stance': -0.8},
  {'subject': 'Russia', 'stance': -1.0},
  {'subject': 'Democracy', 'stance': -0.5},
  {'subject': 'National Sovereignty', 'stance': -0.8},
  {'subject': 'War', 'stance': -0.9}],
 'overall_alignment': 0.8}